In [0]:
from pyspark.sql.functions import lit

In [0]:
date = "2024-12-30"

In [0]:
%run "../includes/configuration"

In [0]:
movies_df = spark.read.parquet(f"{silver_folder_path}/movies") \
    .filter(f"file_date = '{date}' ")

In [0]:
country_df = spark.read.parquet(f"{silver_folder_path}/countries")

In [0]:
production_country_df = spark.read.parquet(f"{silver_folder_path}/productions_countries") \
    .filter(f"created_date = '{date}' ")

In [0]:
production_company_df = spark.read.parquet(f"{silver_folder_path}/productions_companies") \
    .filter(f"file_date = '{date}' ")

In [0]:
movie_company_df = spark.read.parquet(f"{silver_folder_path}/movies_companies") \
    .filter(f"file_date = '{date}' ")

In [0]:
country_prod_coun_df = country_df.join(production_country_df,
                                       country_df.country_id == production_country_df.country_id,
                                       "inner") \
                        .select(country_df.country_name,
                                production_country_df.movie_id)

In [0]:
prod_comp_mov_comp_df = production_company_df.join(movie_company_df,
                                                   production_company_df.company_id == movie_company_df.company_id,
                                                   "inner") \
                        .select(production_company_df.company_name,
                                movie_company_df.movie_id)

In [0]:
movies_filter_df = movies_df.filter("year_release_date >= 2010")

In [0]:
results_movies_country_prod_company_df = movies_filter_df.join(country_prod_coun_df,
                                                               movies_filter_df.movie_id == country_prod_coun_df.movie_id,
                                                               "inner") \
                                                        .join(prod_comp_mov_comp_df,
                                                              movies_filter_df.movie_id == prod_comp_mov_comp_df.movie_id,
                                                              "left")

In [0]:
results_df = results_movies_country_prod_company_df \
    .select("title", "budget", "revenue", "duration_time", "release_date", 
            "country_name", "company_name") \
    .withColumn("created_date", lit(date))

In [0]:
results_order_by_df = results_df.orderBy(results_df.title.asc())

In [0]:
results_order_by_df.write.mode("append").format("parquet").partitionBy("created_date").saveAsTable("movie_gold.results_country_prod_company")

In [0]:
%sql
select *
from movie_gold.results_country_prod_company

In [0]:
results_order_by_df.display()